In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('msv20-21data.txt')
winter2 = pd.read_csv ('msv21-22data.txt')
winter3 = pd.read_csv ('msv22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
6,MSV,2021-12-01 03:22:00,T,UP
8,MSV,2021-12-01 04:56:00,T,UP
9,MSV,2021-12-01 05:56:00,T,UP
10,MSV,2021-12-01 06:12:00,T,UP
37,MSV,2021-12-02 04:13:00,T,-RA
...,...,...,...,...
3148,MSV,2022-02-26 11:47:00,T,-SN BR
3149,MSV,2022-02-26 11:56:00,T,-SN
3150,MSV,2022-02-26 12:12:00,T,-SN
3151,MSV,2022-02-26 12:56:00,T,-SN


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
precip_days3[775:785]

,station,datetime,precip(mm),wxcodes
2073,MSV,2023-01-25 20:13:00,0.0001,-SN BR
2074,MSV,2023-01-25 20:36:00,0.0001,-SN BR
2075,MSV,2023-01-25 20:54:00,0.0001,-SN BR
2076,MSV,2023-01-25 20:55:00,0.0001,-SN BR
2077,MSV,2023-01-25 21:05:00,0.0001,-SN BR
2078,MSV,2023-01-25 21:22:00,0.0001,-SN BR
2079,MSV,2023-01-25 21:33:00,0.0001,-SN BR
2080,MSV,2023-01-25 21:56:00,0.0001,-SN BR
2081,MSV,2023-01-25 22:02:00,0.0001,-SN BR
2082,MSV,2023-01-25 22:15:00,0.25,-SN FZFG


In [9]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 56]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 56]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 56]

In [11]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [12]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [13]:
msv = LocationInfo(
    'Monticello' 'New York', 
    latitude= 41.6556,
    longitude= -74.6893, 
    timezone='America/New_York'
)

msv

LocationInfo(name='MonticelloNew York', region='England', timezone='America/New_York', latitude=41.6556, longitude=-74.6893)

In [14]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(msv.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [15]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(msv.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [16]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(msv.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [17]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:06:58  2020-12-01 16:28:51  2020-12-01
1   2020-12-02 07:08:00  2020-12-02 16:28:37  2020-12-02
2   2020-12-03 07:09:01  2020-12-03 16:28:25  2020-12-03
3   2020-12-04 07:10:00  2020-12-04 16:28:15  2020-12-04
4   2020-12-05 07:10:58  2020-12-05 16:28:08  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:41:01  2021-02-24 17:43:21  2021-02-24
86  2021-02-25 06:39:30  2021-02-25 17:44:34  2021-02-25
87  2021-02-26 06:37:58  2021-02-26 17:45:45  2021-02-26
88  2021-02-27 06:36:25  2021-02-27 17:46:57  2021-02-27
89  2021-02-28 06:34:52  2021-02-28 17:48:08  2021-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:06:43  2021-12-01 16:28:54  2021-12-01
1   2021-12-02 07:07:45  2021-12-02 16:28:39  2021-12-02
2   2021-12-03 07:08:46  2021-12-03 16:28:27  2021-12-03
3   2021-12-04 07:09:46  2021-12-04 16:28:17  2021-12-04
4   2021-12-05 07:10:44  2021-12-05 16:28:09  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:41:23  2022-02-24 17:43:04  2022-02-24
86  2022-02-25 06:39:52  2022-02-25 17:44:16  2022-02-25
87  2022-02-26 06:38:20  2022-02-26 17:45:28  2022-02-26
88  2022-02-27 06:36:48  2022-02-27 17:46:40  2022-02-27
89  2022-02-28 06:35:14  2022-02-28 17:47:51  2022-02-28

[90 rows x 3 columns]


In [19]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:06:28  2022-12-01 16:28:58  2022-12-01
1   2022-12-02 07:07:30  2022-12-02 16:28:43  2022-12-02
2   2022-12-03 07:08:31  2022-12-03 16:28:29  2022-12-03
3   2022-12-04 07:09:31  2022-12-04 16:28:19  2022-12-04
4   2022-12-05 07:10:30  2022-12-05 16:28:10  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:41:45  2023-02-24 17:42:46  2023-02-24
86  2023-02-25 06:40:14  2023-02-25 17:43:58  2023-02-25
87  2023-02-26 06:38:43  2023-02-26 17:45:10  2023-02-26
88  2023-02-27 06:37:10  2023-02-27 17:46:22  2023-02-27
89  2023-02-28 06:35:37  2023-02-28 17:47:34  2023-02-28

[90 rows x 3 columns]


In [20]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [21]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MSV,2020-12-01 00:56:00,0.5100,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
1,MSV,2020-12-01 01:56:00,1.0200,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
2,MSV,2020-12-01 02:56:00,1.5200,RA BR,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
3,MSV,2020-12-01 03:56:00,0.5100,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
4,MSV,2020-12-01 05:56:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
...,...,...,...,...,...,...,...
607,MSV,2021-02-27 13:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
608,MSV,2021-02-27 14:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
609,MSV,2021-02-27 15:56:00,0.2500,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
610,MSV,2021-02-27 16:56:00,0.0001,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57


In [22]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MSV,2021-12-01 04:56:00,0.0001,UP,2021-12-01,2021-12-01 07:06:43,2021-12-01 16:28:54
1,MSV,2021-12-01 05:56:00,0.0001,UP,2021-12-01,2021-12-01 07:06:43,2021-12-01 16:28:54
2,MSV,2021-12-02 04:56:00,0.0001,-SN BR,2021-12-02,2021-12-02 07:07:45,2021-12-02 16:28:39
3,MSV,2021-12-02 05:56:00,0.5100,-SN BR,2021-12-02,2021-12-02 07:07:45,2021-12-02 16:28:39
4,MSV,2021-12-02 06:56:00,1.0200,-RA BR,2021-12-02,2021-12-02 07:07:45,2021-12-02 16:28:39
...,...,...,...,...,...,...,...
420,MSV,2022-02-26 09:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:38:20,2022-02-26 17:45:28
421,MSV,2022-02-26 10:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:38:20,2022-02-26 17:45:28
422,MSV,2022-02-26 11:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:38:20,2022-02-26 17:45:28
423,MSV,2022-02-26 12:56:00,0.0001,-SN,2022-02-26,2022-02-26 06:38:20,2022-02-26 17:45:28


In [23]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MSV,2022-12-01 06:56:00,0.0001,UP,2022-12-01,2022-12-01 07:06:28,2022-12-01 16:28:58
1,MSV,2022-12-01 09:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:06:28,2022-12-01 16:28:58
2,MSV,2022-12-01 11:56:00,0.0001,-SN BR,2022-12-01,2022-12-01 07:06:28,2022-12-01 16:28:58
3,MSV,2022-12-01 12:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:06:28,2022-12-01 16:28:58
4,MSV,2022-12-01 13:56:00,0.0001,-SN,2022-12-01,2022-12-01 07:06:28,2022-12-01 16:28:58
...,...,...,...,...,...,...,...
444,MSV,2023-02-25 16:56:00,0.5100,-SN,2023-02-25,2023-02-25 06:40:14,2023-02-25 17:43:58
445,MSV,2023-02-25 17:56:00,0.2500,-SN,2023-02-25,2023-02-25 06:40:14,2023-02-25 17:43:58
446,MSV,2023-02-25 18:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:40:14,2023-02-25 17:43:58
447,MSV,2023-02-25 19:56:00,0.0001,-SN,2023-02-25,2023-02-25 06:40:14,2023-02-25 17:43:58


In [24]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
9,MSV,2020-12-02 08:56:00,0.0001,UP,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37
10,MSV,2020-12-02 09:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37
11,MSV,2020-12-02 10:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37
12,MSV,2020-12-02 11:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37
13,MSV,2020-12-02 12:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37
...,...,...,...,...,...,...,...
606,MSV,2021-02-27 12:56:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
607,MSV,2021-02-27 13:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
608,MSV,2021-02-27 14:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57
609,MSV,2021-02-27 15:56:00,0.2500,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57


In [25]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
9,MSV,2020-12-02 08:56:00,0.0001,UP,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37,2020-12-02 08:08:00,2020-12-02 15:28:37
10,MSV,2020-12-02 09:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37,2020-12-02 08:08:00,2020-12-02 15:28:37
11,MSV,2020-12-02 10:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37,2020-12-02 08:08:00,2020-12-02 15:28:37
12,MSV,2020-12-02 11:56:00,0.0001,-SN,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37,2020-12-02 08:08:00,2020-12-02 15:28:37
13,MSV,2020-12-02 12:56:00,0.0001,-SN BR,2020-12-02,2020-12-02 07:08:00,2020-12-02 16:28:37,2020-12-02 08:08:00,2020-12-02 15:28:37
...,...,...,...,...,...,...,...,...,...
605,MSV,2021-02-27 11:56:00,0.2500,-SN BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57,2021-02-27 07:36:25,2021-02-27 16:46:57
606,MSV,2021-02-27 12:56:00,0.5100,-SN BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57,2021-02-27 07:36:25,2021-02-27 16:46:57
607,MSV,2021-02-27 13:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57,2021-02-27 07:36:25,2021-02-27 16:46:57
608,MSV,2021-02-27 14:56:00,1.0200,UP BR,2021-02-27,2021-02-27 06:36:25,2021-02-27 17:46:57,2021-02-27 07:36:25,2021-02-27 16:46:57


In [27]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [28]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [29]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.2360519650655022
2021-2022: 0.4236910828025479
2022-2023: 0.32115546875


In [30]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,MSV,2020-12-01 00:56:00,0.5100,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
1,MSV,2020-12-01 01:56:00,1.0200,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
2,MSV,2020-12-01 02:56:00,1.5200,RA BR,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
3,MSV,2020-12-01 03:56:00,0.5100,-RA,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
4,MSV,2020-12-01 05:56:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:06:58,2020-12-01 16:28:51
...,...,...,...,...,...,...,...
596,MSV,2021-02-24 00:56:00,0.0001,-RA,2021-02-24,2021-02-24 06:41:01,2021-02-24 17:43:21
597,MSV,2021-02-24 02:56:00,0.0001,-RA,2021-02-24,2021-02-24 06:41:01,2021-02-24 17:43:21
598,MSV,2021-02-25 01:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:39:30,2021-02-25 17:44:34
599,MSV,2021-02-25 02:56:00,0.0001,-RA,2021-02-25,2021-02-25 06:39:30,2021-02-25 17:44:34


In [31]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_25972\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [32]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [33]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [34]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.113144
2021-2022: 0.30574772727272725
2022-2023: 0.46064814814814814


In [35]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('msv.csv', index=False)